In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import requests
import json
import time
import hashlib
from time import sleep

In [ ]:
YC_FOLDER_ID = ###
YC_API_KEY = ###

In [ ]:
class Agent:
    """
    This class represents a single focus group participant with personality parameters
    and language model access to generate personalized responses.
    """

    def __init__(self, avatar_for_agent, api_key, model_name):
        self.name = avatar_for_agent.get("name", "")
        self.age = avatar_for_agent.get("age", None)
        self.profession = avatar_for_agent.get("profession", "")
        self.sentiment = avatar_for_agent.get("sentiment", 0.5)
        self.traits = avatar_for_agent.get("traits", "")
        self.concerns = avatar_for_agent.get("concerns", "")
        self.key_phrases = avatar_for_agent.get("key_phrases", "")
        self.communication_style = avatar_for_agent.get("communication_style", "")
        self.economic_view = avatar_for_agent.get("economic_view", "")
        self.trust_in_institutions = avatar_for_agent.get("trust_in_institutions", 0.5)
        self.knowledge_level = avatar_for_agent.get("knowledge_level", "")
        self.financial_behavior = avatar_for_agent.get("financial_behavior", "")
        self.policy_priority = avatar_for_agent.get("policy_priority", "")
        self.cb_functions_understanding = avatar_for_agent.get("cb_functions_understanding", "")
        self.cb_perception = avatar_for_agent.get("cb_perception", "")
        self.cb_trust_factors = avatar_for_agent.get("cb_trust_factors", "")
        self.information_sources = avatar_for_agent.get("information_sources", "")
        self.emotional_tone = avatar_for_agent.get("emotional_tone", "")
        self.api_key = api_key
        self.model_name = model_name
        self.trust_history = [self.trust_in_institutions]  # Track trust history
        self.response_history = []  # Store response text
        self.amenable_to_influence = self.calculate_amenability(self.trust_in_institutions)

    def calculate_amenability(self, trust):
        """
        Calculate influence amenability based on trust_in_institutions
        - Close to 0 or 1: low influence (0)
        - 0.3-0.5 range: high influence (1)
        """
        if trust < 0.3:
            return trust / 0.6  # Linear increase from 0 to 0.5
        elif trust <= 0.5:
            return 1.0  # Plateau at maximum influence
        else:
            return (1.0 - trust) / 1  # Linear decrease from 0.5 to 0

    def is_complete_response(self, text: str) -> bool:
        """More accurate response completion check"""
        # Check minimum length threshold
        if len(text) < 30:  # Increased minimum length
            return False

        # Check for proper ending punctuation
        if not any(text.endswith(punc) for punc in ['.', '!', '?', '"', "'", "»"]):
            # Allow parentheses if they're balanced
            # if text.count('(') != text.count(')'):
            #     return False
            # Allow lists/dashes
            if not any(text.endswith(c) for c in [':', '-', '—', '*']):
                return False
        return True

    def generate_response(self, prompt, group_opinions=None, retry_count=0, max_output_tokens=1000):
        """
        This method generates a text response to the given prompt, personalized
        based on the agent's traits and profile.
        """

        MAX_RETRIES = 3

        full_prompt = (
            f"Ты — участник фокус-группы по имени {self.name}, {self.age} лет, "
            f"{self.profession}. Твой характер отличается следующими чертами ({self.traits}), "
            f"и тебя беспокоит следующее: {self.concerns}. "
            f"Ты выражаешь мысли через {self.communication_style}, и твои ключевые фразы — это: {self.key_phrases}. "
            f"Твоя точка зрения на экономику: {self.economic_view}. "
            f"Уровень доверия к институтам: {self.trust_in_institutions}, а уровень знаний — {self.knowledge_level}. "
            f"Ты ведёшь себя так: {self.financial_behavior}, приоритет в политике — {self.policy_priority}. "
            f"Ты понимаешь функции ЦБ как: {self.cb_functions_understanding}, но воспринимаешь его как: {self.cb_perception}, "
            f"доверяешь только при наличии: {self.cb_trust_factors}. "
            f"Ты получаешь информацию из: {self.information_sources}. "
            f"Твой эмоциональный тон: {self.emotional_tone}. "
            f"Вопрос, на который ты отвечаешь: {prompt}"
            f"Если вопрос содержит указания на другие мнения, учти их при формулировке ответа, если они согласуются с твоими убеждениями."
        )

        if group_opinions:
            influence_context = (
                f"\nКонтекст обсуждения: Другие участники выразили мнения: {group_opinions}. "
                f"Твоя восприимчивость к мнениям других: {self.amenable_to_influence} "
                f"(0 - невосприимчив, 1 - очень восприимчив). Учти это при формировании ответа."
            )
            full_prompt += influence_context

        stop_sequences = ["\n\n"]

        url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"

        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Api-Key {YC_API_KEY}",
            "x-folder-id": YC_FOLDER_ID,
        }

        payload = {
            "modelUri": f"gpt://{YC_FOLDER_ID}/yandexgpt",
            "completionOptions": {
                "stream": False,
                "temperature": self.sentiment if self.sentiment > 0 else 0.1,
                "maxTokens": 2000
            },
            "messages": [
                {
                    "role": "system",
                    "text": full_prompt
                },
                {
                    "role": "user",
                    "text": f"Вопрос: {prompt}"
                }
            ]
        }

        response_content = ""
        try:
            with requests.post(url, headers=headers, json=payload) as response:
                result = response.json()
                response_content = result["result"]["alternatives"][0]["message"]["text"]

        except Exception as e:
            print(f"Error in Yandex GPT request: {e}")
            response_content = "Произошла ошибка при обращении к сервису генерации ответов."

        # Обновляем trust и историю ответов
        self.update_trust(response_content)
        self.response_history.append(response_content)

        return response_content



    # Trust shifting
    def update_trust(self, response):
        """Analyze response sentiment and update agent's trsut"""
        # Simple rule-based sentiment analysis (customize as needed)
        positive_triggers = ["согласен", "поддержив", "оптимистич", "позитив", "одобряю",
                             "разделяю", "довер", "понимаю", "понят", "уверен",
                             "стабильн", "успешн", "прост", "ясн", "хорош", "да", "больш", "положитель"]
        negative_triggers = ["не согласен", "поддержив", "критич", "скептич", "опасен", "негатив",
                             "не разделяю", "не довер", "не понимаю", "не понят",
                             "сомнен", "риск", "сложн", "недостаточн", "нет", "меньш"]

        # Count sentiment indicators
        import re
        lower_text = response.lower()

        pos_count = sum(len(re.findall(rf'\b{trigger}\b', lower_text))
                    for trigger in positive_triggers)

        neg_count = sum(len(re.findall(rf'\b{trigger}\b', lower_text))
                    for trigger in negative_triggers)

        # Calculate sentiment shift (customize weights as needed)
        shift = (pos_count * 0.02) - (neg_count * 0.02)

        # Apply shift with bounds [0, 1]
        new_trust = max(0, min(1, self.trust_in_institutions + shift))

        # Update and track
        self.trust_in_institutions = new_trust
        self.trust_history.append(new_trust)

        return new_trust

    def calibrate(self, calibration_path, max_delay=60):
        """Calibrate agent using documents in specified directory"""
        if not os.path.exists(calibration_path):
            raise FileNotFoundError(f"Calibration directory not found: {calibration_path}")

        calibration_results = []
        file_list = sorted(os.listdir(calibration_path))

        VALID_RESPONSES = {"негативно", "нейтрально", "позитивно"}

        for i, filename in enumerate(file_list):
            file_path = os.path.join(calibration_path, filename)
            if not os.path.isfile(file_path) or not filename.endswith(".txt"):
                continue

            with open(file_path, 'r', encoding='utf-8') as file:
                document = file.read()

                prompt = (
                    "Оцени этот текст с учетом следующих факторов:\n"
                    "Обоснованность принимаемого решения\n"
                    "Полнота объяснений\n"
                    "Доступность языка документа\n"
                    "Способность документа вызвать доверие\n\n"
                    "ТВОЙ ОТВЕТ ДОЛЖЕН БЫТЬ ТОЛЬКО ОДНИМ СЛОВОМ ИЗ СПИСКА:\n"
                    "['негативно', 'нейтрально', 'позитивно']\n\n"
                    "НЕ ДОБАВЛЯЙ НИКАКИХ ДРУГИХ СЛОВ, КОММЕНТАРИЕВ ИЛИ ЗНАКОВ ПРЕПИНАНИЯ.\n"
                    f"Текст документа:\n{document}"
                )

                # Dynamic delay based on request count
                base_delay = min(1 * (i // 10), max_delay)
                time.sleep(base_delay)

                # First try with tight token limit
                response = self.generate_response(prompt, max_output_tokens=15)

                # If response is invalid, try again with more tokens
                clean_response = response.strip().lower()
                if clean_response not in VALID_RESPONSES:
                    print(f"Invalid response '{response}'. Retrying with more tokens...")
                    response = self.generate_response(prompt, max_output_tokens=15)
                    clean_response = response.strip().lower()
                calibration_results.append({
                    "document": filename,
                    "response": response.strip().lower()
                })

                # Pause every 10 documents
                if i % 10 == 9:
                    time.sleep(5)

        return calibration_results


class FocusGroup:
    """
    This class creates & manages a group of agents and provides tools to interact with them as a unit.
    """
    def __init__(self, interviews_path: str = "interviews",
                 avatars_path: str = "avatars",
                 result_path: str = "result.txt",
                 model_for_preprocessing: str = "deepseek/deepseek-r1",
                 api_key_preprocessing: str = "",
                 use_auto_preprocessing: bool = True,
                #  model_for_agent = None,
                #  tokenizer_for_agent = None,
                 api_key_agent: str = "",  # новый параметр
                 model_name_agent: str = "deepseek/deepseek-r1",  # новый параметр
                 iterations: int = 2):
        """
        This method initializes all agents in the focus group from a list of config dicts.
        The resulting number of agents will be equal to the number of downloaded interviews.
        Parameters:
        interviews_path -> path to the directory with the interviews ("/interviews" by default)
        avatars_path -> path to the directory where ready-to-use avatars will be stored in json format ("/avatars")
        result_path -> pathto the file we want to store the results of interview in ("/result.txt" by default)
        model_for_preprocessing -> name of the model in OpenRouter (DeepSeek R1 by default)
        api_key_preprocessing -> API key for OpenRouter
        use_auto_preprocessing -> whether we like our interviews preprocessed automatically (via OpenRouter API, True by default), if false, avatars are downloaded manually
        model_for_agent -> any pretrained model downloaded locally via transformers (qwen, mistral etc.)
        tokenizer_for_agent -> tokenizer for pretrained model
        iterations -> NB! -> parameter that sets number of times the questions will be processed by each agent
        """
        self.interviews_path = interviews_path
        self.avatars_path = avatars_path
        self.result_path = result_path
        self.model_for_preprocessing = model_for_preprocessing
        self.api_key_preprocessing = api_key_preprocessing
        self.use_auto_preprocessing = use_auto_preprocessing
        # self.model_for_agent = model_for_agent
        # self.tokenizer_for_agent = tokenizer_for_agent
        self.iterations = iterations
        self.api_key_agent = api_key_agent
        self.model_name_agent = model_name_agent
        self.last_request_time = time.time()
        self.request_count = 0

        if self.use_auto_preprocessing: # if we want avatars created automatically
            self.make_avatars()
        avatar_files = [f for f in os.listdir(self.avatars_path) if os.path.isfile(os.path.join(self.avatars_path, f))]
        avatars = []
        for avatar_file in avatar_files:
            with open(os.path.join(self.avatars_path, avatar_file), "r", encoding="utf-8") as file:
                avatar = json.load(file)
                avatars.append(avatar)
        self.avatars = avatars
        self.agents = []
        # for i, avatar_for_agent in enumerate(avatars):
        #     agent = self.create_agent(avatar_for_agent, self.model_for_agent, self.tokenizer_for_agent)
        #     self.agents.append(agent)
        for avatar in avatars:
            agent = Agent(avatar, self.api_key_agent, self.model_name_agent)
            self.agents.append(agent)


    def make_avatars(self):

        def process_content(content):
            """cleans the content of the automatically generated tags"""
            return content.replace('<think>', '').replace('</think>', '')

        SYSTEM_PROMPT = """Ты — профессиональный аналитик социологических и экономических исследований.\n
                        Твоя задача — анализировать текстовые расшифровки интервью с респондентами и извлекать ключевые характеристики в строго заданном JSON-формате.\n
                        **Правила обработки:**\n
                        1. Входные данные: Строка (str) с расшифровкой интервью на русском языке.\n
                        2. Анализируй только явно упомянутую информацию и логические выводы из ответов.\n
                        3. Количественные оценки (sentiment, trust) вычисляй по шкале 0.0-1.0 на основе:\n
                        - Лексического анализа (эмоциональные маркеры, модальность)\n
                        - Косвенных индикаторов (уверенность/неуверенность в суждениях)\n
                        4. Для текстовых полей (traits, concerns и др.) используй:\n
                        - Цитаты из интервью (если точно соответствуют)\n
                        - Краткие обобщения (3-5 слов на пункт)\n
                        5. Уровень знаний определяй по:\n
                        - Использованию терминологии\n
                        - Глубине объяснений\n
                        - Упоминанию образования/опыта\n
                        **Требуемый JSON-формат:**\n
                        {\n
                        "name": "Имя из текста (англ. форма)",\n
                        "age": "Цифра (возраст)",\n
                        "profession": "Основной род занятий (до 3 слов)",\n
                        "sentiment": "Float (0.0-1.0) общий позитив тональности",\n
                        "traits": "Черты характера через запятую (4-5 ключевых)",\n
                        "concerns": "Главные экономические тревоги через запятую",\n
                        "key_phrases": "Дословные цитаты (2-3 реплики, разделенные '|')",\n
                        "communication_style": "Стилистика речи (2-3 характеристики)",\n
                        "economic_view": "Суть взглядов на экономику (1 предложение)",\n
                        "trust_in_institutions": "Float (0.0-1.0) доверие к гос.институтам",\n
                        "knowledge_level": "Оценка компетентности (формат: 'Уровень' ('пояснение'))",\n
                        "financial_behavior": "Паттерны поведения (2-3 ключевых)",\n
                        "policy_priority": "Главный экономический приоритет (1 пункт)",\n
                        "cb_functions_understanding": "Понимание функций ЦБ (3 пункта через запятую)",\n
                        "cb_perception": "Отношение к ЦБ (1 предложение)",\n
                        "cb_trust_factors": "Факторы доверия к ЦБ (2-3 пункта)",\n
                        "information_sources": "Источники инфо через запятую",\n
                        "emotional_tone": "Доминирующая эмоция (1-2 слова)"\n
                        }\n
                        **Критические инструкции:**\n
                        - Если данные отсутствуют: используй null (кроме текстовых полей - оставляй пустую строку).\n
                        - Для оценок (sentiment/trust): 0.8+ = явное доверие/позитив, 0.6-0.79 = умеренное, <0.6 = скепсис.\n
                        - knowledge_level: "Низкий", "Базовый", "Средний", "Средне-высокий", "Высокий".\n
                        - В key_phrases включай ТОЛЬКО дословные цитаты (макс. 7 слов).\n
                        - В cb_perception укажи когнитивную глубину (глубокое/поверхностное понимание).\n
                        - Учитывай контекстные противоречия (напр., "доверяю, но не разбираюсь" → умеренное доверие)."""

        headers = {
            "Authorization": f"Bearer {self.api_key_preprocessing}",
            "Content-Type": "application/json"
            }
        txt_files = [f for f in os.listdir(self.interviews_path) if f.endswith('.txt')]
        interviews = []
        if not txt_files:
            print("В папке нет .txt файлов!")
            return
        for file_name in txt_files:
            file_path = os.path.join(self.interviews_path, file_name)
            with open(file_path, 'r', encoding='utf-8') as file:
                  content = file.read()
                  interviews.append(content)
        for interview in interviews:
              messages = [{"role": "system", "content": SYSTEM_PROMPT}, {"role": "user", "content": interview}]
              data = {
              "model": self.model_for_preprocessing,
              "messages": messages
              }
              try:
                  response = requests.post("https://openrouter.ai/api/v1/chat/completions",
                  headers=headers,
                  json=data,
                  timeout=30
                  )
                  response_json = response.json()
                  content_str = response_json['choices'][0]['message']['content']
                  avatar_data = json.loads(content_str)
                  filename = f"{avatar_data['name']}.json"
                  with open(os.path.join(self.avatars_path, filename), 'w') as f:
                      json.dump(avatar_data, f)
              except Exception as e:
                  print(f"Ошибка API: {str(e)}")
                  continue


    def create_agent(self, avatar_for_agent, model, tokenizer):
        """
        This function creates a single Agent instance based on config and model.
        """
        return Agent(avatar_for_agent, model, tokenizer)


    def get_group_number(self):
        """
        This method returns the total number of agents in the group.
        """
        return len(self.agents)

    def calibrate_all(self, calibration_path=None, save_path=r"calibration_results.json"):
        """Calibrate all agents and save results to JSON"""
        if not calibration_path:
            return {}

        calibration_results = {}
        for agent in self.agents:
            try:
                agent_results = agent.calibrate(calibration_path)
                calibration_results[agent.name] = {
                    "profile": {
                        "age": agent.age,
                        "profession": agent.profession,
                        "trust_level": agent.trust_in_institutions
                    },
                    "responses": agent_results
                }
                # Save to JSON with proper formatting
                with open(save_path, 'w', encoding='utf-8') as f:
                    json.dump(calibration_results, f,
                            ensure_ascii=False,
                            indent=2,
                            default=str)  # Handle non-serializable objects

            except Exception as e:
                print(f"Calibration failed for {agent.name}: {str(e)}")
                calibration_results[agent.name] = {"error": str(e)}

        # # Save to JSON with proper formatting
        # with open(save_path, 'w', encoding='utf-8') as f:
        #     json.dump(calibration_results, f,
        #             ensure_ascii=False,
        #             indent=2,
        #             default=str)  # Handle non-serializable objects

        print(f"Calibration results saved to {save_path}")
        return calibration_results


    def respond_all(self, prompt):
        """
        This method collects responses from all agents to a given prompt.
        """
        # responses = []
        # for agent in self.agents:
        #     responses.append((agent.name, agent.generate_response(prompt)))
        # return responses

        """
        Collect responses for all iterations in proper order
        Returns: list of tuples (agent_name, response, iteration_number)
        """
        all_responses = []

        # First iteration
        iteration_context = f"Вопрос группе: {prompt}"
        all_responses.append(("SYSTEM", iteration_context, 0))

        iteration_responses = []
        for agent in self.agents:
            response = agent.generate_response(prompt)
            print(f"Got response from {agent.name}: {response}")
            response_tuple = (agent.name, response, 1)
            iteration_responses.append(response_tuple)
            all_responses.append(response_tuple)

        # Subsequent iterations
        for iteration in range(2, self.iterations + 1):
            # Build summary from previous iteration
            prev_iteration_responses = [resp for (_, resp, it) in iteration_responses if it == iteration-1]
            summary_text = '; '.join([f"{name}: {resp[:500]}..."
                                    for (name, resp, it) in iteration_responses
                                    if it == iteration-1])

            # Create new prompt with context
            new_prompt = f"Ответь на вопрос: {prompt}. При формулировке ответа учти следующие мнения: {summary_text}"

            # Store context for output
            iteration_context = f"Вопрос задан группе повторно (Итерация {iteration}): {new_prompt}"
            all_responses.append(("SYSTEM", iteration_context, 0))

            # Clear previous iteration and collect new responses
            iteration_responses = []
            for agent in self.agents:
                response = agent.generate_response(new_prompt, group_opinions=summary_text)
                response_tuple = (agent.name, response, iteration)
                iteration_responses.append(response_tuple)
                all_responses.append(response_tuple)
        return all_responses


    def start_dialog_auto(self, questions_file):
        """
        This function interacts with the user, accepts questions, sends them to all agents,
        and prints & saves responses until user types 'stop'.
        """
        print(f"Your synthetic focus group consists of {len(self.agents)} respondents.")
        previous_trust = {agent.name: agent.trust_in_institutions for agent in self.agents}

        with open(questions_file, "r") as f:
            questions = f.readlines()

        for num, question in enumerate(questions):
            print(f"\nВопрос {num}:", question)
            pre_question_trust = {agent.name: agent.trust_in_institutions for agent in self.agents}
            responses = self.respond_all(question)
            with open(self.result_path, 'a', encoding='utf-8', errors='replace') as file:
                for i, response in enumerate(responses):
                    name, content, iteration = response
                    #print(f"  Writing response {i+1}/{len(responses)} for {name}")
                    try:
                        if name == "SYSTEM":
                            file.write(f"{content}\n")
                        else:
                            file.write(f"{name} ответил: {content}\n")
                        file.flush()
                        os.fsync(file.fileno())
                    except Exception as e:
                        print(f"Error writing response: {str(e)}")
                        # Write error-safe version
                        safe_content = content.encode('utf-8', 'replace').decode('utf-8')
                        file.write(f"{name} ответил: [ERROR] {safe_content[:500]}\n")

            # Print trust changes
            '''
            print("\nИзменения доверия к институтам:")
            for agent in self.agents:
                change = agent.trust_in_institutions - pre_question_trust[agent.name]
                print(f"{agent.name}: {change:+.2f} (Было: {pre_question_trust[agent.name]:.2f}, Стало: {agent.trust_in_institutions:.2f})")
'''
        # Save full history when exiting
        with open("trust_history.json", "w", encoding='utf-8') as f:
            json.dump({
                agent.name: {
                    "trust_history": agent.trust_history,
                    "responses": agent.response_history,
                    "influence_factor": agent.amenable_to_influence,
                    "final_trust": agent.trust_in_institutions
                } for agent in self.agents
            }, f, ensure_ascii=False, indent=2)

if __name__ == "__main__":
    API = "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" # "#########################################################################"
    group_initial = FocusGroup(
        avatars_path=r'params',
        use_auto_preprocessing = False,
        api_key_preprocessing=API,
        api_key_agent=API,  # передаем ключ для агентов
        model_name_agent="deepseek/deepseek-r1", #"deepseek/deepseek-r1-0528:free", # "deepseek/deepseek-r1", # "deepseek/deepseek-r1:free", # "mistralai/mistral-7b-instruct:free", # "mistralai/mistral-7b-instruct", # "deepseek/deepseek-r1",  # или другая модель
        # model_for_agent = model,
        # tokenizer_for_agent = tokenizer,
        iterations = 2 # 2
    )
    group_initial.start_dialog_auto("questions_initial.txt")


Your synthetic focus group consists of 10 respondents.

Вопрос 0: ДИНАМИКА ПОТРЕБИТЕЛЬСКИХ ЦЕН: ФАКТЫ, ОЦЕНКИ, КОММЕНТАРИИ (АПРЕЛЬ 2025 Г.). В апреле месячный рост цен с исключением сезонности продолжил замедляться. Показатели устойчивого ценового давления были преимущественно ниже, чем в марте. Вклад волатильных компонентов, напротив, возрос. При этом динамика цен остается весьма разнородной по группам товаров и услуг. Непродовольственные товары в среднем за месяц показали околонулевой прирост цен. В то же время рост цен на продовольствие и услуги оставался высоким. По прогнозу Банка России, с учетом проводимой денежно-кредитной политики годовая инфляция снизится до 7,0–8,0% в 2025 г. и вернется к 4,0% в 2026 году. •В апреле 2025 г., по данным Росстата, потребительские цены выросли на 0,40% (в марте 2025 г. – на 0,65%). С сезонной корректировкой в годовом выражении (с.к.г.) прирост цен составил 6,2% (в марте – 7,0%, в 1к25 – 8,2%, в 4к24 – 12,9%, в 3к24 – 12,1%, в 2к24 – 8,0%). Годова

In [ ]:
    group_amended = FocusGroup(
        avatars_path=r'params',
        use_auto_preprocessing = False,
        api_key_preprocessing=API,
        api_key_agent=API,  # передаем ключ для агентов
        model_name_agent="deepseek/deepseek-r1", #"deepseek/deepseek-r1-0528:free", # "deepseek/deepseek-r1", # "deepseek/deepseek-r1:free", # "mistralai/mistral-7b-instruct:free", # "mistralai/mistral-7b-instruct", # "deepseek/deepseek-r1",  # или другая модель
        # model_for_agent = model,
        # tokenizer_for_agent = tokenizer,
        iterations = 2 # 2
    )
    group_amended.start_dialog_auto("questions_amended.txt")

Your synthetic focus group consists of 10 respondents.

Вопрос 0: ДИНАМИКА ПОТРЕБИТЕЛЬСКИХ ЦЕН: ФАКТЫ, ОЦЕНКИ, КОММЕНТАРИИ (АПРЕЛЬ 2025 Г.) В апреле месячный рост цен (без учета сезонных колебаний, таких как новогодний спрос) продолжил замедляться. Показатели устойчивого ценового давления, т.е. устойчивого роста цен были преимущественно ниже, чем в марте. Вклад волатильных (нестабильных) компонентов, напротив, возрос. При этом динамика цен остается весьма разнородной по группам товаров и услуг. Непродовольственные товары в среднем за месяц почти не показали роста цен. В то же время рост цен на продовольствие и услуги оставался высоким. По оценке Банка России, с учетом проводимой им денежно-кредитной политики годовая инфляция снизится до 7,0–8,0% в 2025 г. и вернется к 4,0% в 2026 году. •В апреле 2025 г., по данным Росстата, потребительские цены выросли на 0,40% (для сравнения: в марте 2025 г. – на 0,65%). С сезонной корректировкой в годовом выражении (с.к.г., то есть, как если бы текущ